# Sample Lya posterior as a function of nuisance parameters

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
import os
import json
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 120
mpl.rcParams['figure.dpi'] = 120
import cProfile
import emcee
# our own modules
import simplest_emulator
import linear_emulator
import data_PD2013
import mean_flux_model
import thermal_model
import lya_theory
import likelihood

### Setup Lya likelihood function

In [2]:
# read P1D measurement
data=data_PD2013.P1D_PD2013(blind_data=True)
zs=data.z

In [3]:
# load P1D emulator
emu=linear_emulator.LinearEmulator(verbose=False)

setup interpolator for coefficient 0
[ 0.34215808 -2.30000012  0.64144805  0.14339793  1.47513694] test [-1.18323575]
setup interpolator for coefficient 1
[ 0.34215808 -2.30000012  0.64144805  0.14339793  1.47513694] test [-0.6323589]
setup interpolator for coefficient 2
[ 0.34215808 -2.30000012  0.64144805  0.14339793  1.47513694] test [-0.29370814]
setup interpolator for coefficient 3
[ 0.34215808 -2.30000012  0.64144805  0.14339793  1.47513694] test [-0.1279153]
setup interpolator for coefficient 4
[ 0.34215808 -2.30000012  0.64144805  0.14339793  1.47513694] test [-0.01945575]


In [39]:
# setup theory to predict Lya 1D power
theory = lya_theory.LyaTheory(zs,emulator=emu,verbose=True)
# for this simple test, just use fiducial cosmology
theory.set_cosmo_model(linP_model=theory.cosmo.linP_model_fid)
linP_Mpc_params=theory.cosmo.get_linP_Mpc_params()

Note: redshifts have been re-sorted (earliest first)
use default mean flux model
use default thermal model


In [43]:
# specify free parameters in likelihood
free_parameters=['ln_tau_0','Delta2_star']
#free_parameters=['ln_tau_0','ln_tau_1']
#free_parameters=['ln_tau_0','ln_tau_1','ln_T0_0']
#free_parameters=['ln_tau_0','ln_tau_1','ln_T0_0','ln_gamma_0']
#free_parameters=['ln_tau_0','ln_tau_1','ln_T0_0','ln_T0_1','ln_gamma_0','ln_gamma_1']

In [44]:
like=likelihood.Likelihood(data=data,theory=theory,free_parameters=free_parameters,verbose=True)

got parameters
g_star = 0.9677508579459803
f_star = 0.98136955784
Delta2_star = 0.360175905286
n_star = -2.29933566726
alpha_star = -0.216527037121
ln_tau_0 = -0.794580172414
ln_tau_1 = 3.18
ln_T0_0 = 9.21034037198
ln_T0_1 = 0.0
ln_gamma_0 = 0.336472236621
ln_gamma_1 = 0.0
par g_star = 0.9677508579459803
fixed
par f_star = 0.98136955784
fixed
par Delta2_star = 0.360175905286
free
par n_star = -2.29933566726
fixed
par alpha_star = -0.216527037121
fixed
par ln_tau_0 = -0.794580172414
free
par ln_tau_1 = 3.18
fixed
par ln_T0_0 = 9.21034037198
fixed
par ln_T0_1 = 0.0
fixed
par ln_gamma_0 = 0.336472236621
fixed
par ln_gamma_1 = 0.0
fixed
likelihood setup with 2 free parameters
2 free parameters


In [45]:
for p in like.free_params:
    print(p.name,p.value)

Delta2_star 0.360175905286
ln_tau_0 -0.794580172414


In [46]:
like.go_silent()

### Setup MCMC to call this function

In [47]:
def log_prob(values,like,linP_Mpc_params,verbose=False):
    test_log_prob=like.log_prob(values=values,linP_Mpc_params=linP_Mpc_params)
    if np.isnan(test_log_prob):
        if verbose:
            print('parameter values outside hull',values)
        return -np.inf
    return test_log_prob

In [48]:
def setup_walkers(ndim,nwalkers,like,linP_Mpc_params,verbose=False):
    print('setup %d walkers with %d dimensions'%(nwalkers,ndim))
    p0 = np.random.rand(ndim * nwalkers).reshape((nwalkers, ndim))
    # make sure that all walkers are within the convex hull
    for iw in range(nwalkers):
        walker=p0[iw]
        if verbose: print(iw,'walker',walker)
        test=log_prob(walker,like,linP_Mpc_params)
        while (test == -np.inf):
            if verbose: print(iw,'bad walker',walker)
            walker = np.random.rand(ndim)
            if verbose: print(iw,'try walker',walker)
            test=log_prob(walker,like,linP_Mpc_params)
        if verbose: print(iw,'good walker',walker,' log_prob=',test)
        p0[iw]=walker
    return p0

In [49]:
# for now we'll have only three dimensions (mean flux, temperature, gamma)
ndim=len(like.free_params)
# setup initial walkers
nwalkers = 20

In [50]:
p0=setup_walkers(ndim,nwalkers,like,linP_Mpc_params)
p0

setup 20 walkers with 2 dimensions
Delta2_star = 0.377395794668 updated 0.377395794668
Delta2_star = 0.439204985912 updated 0.439204985912
Delta2_star = 0.363181204043 updated 0.363181204043
Delta2_star = 0.39145433607 updated 0.39145433607
Delta2_star = 0.478353682974 updated 0.478353682974
Delta2_star = 0.376509062805 updated 0.376509062805
Delta2_star = 0.327824692543 updated 0.327824692543
Delta2_star = 0.490655954803 updated 0.490655954803
Delta2_star = 0.313518256923 updated 0.313518256923
Delta2_star = 0.493140339774 updated 0.493140339774
Delta2_star = 0.309067614221 updated 0.309067614221
Delta2_star = 0.44944190211 updated 0.44944190211
Delta2_star = 0.355896899486 updated 0.355896899486
Delta2_star = 0.315397211784 updated 0.315397211784
Delta2_star = 0.429513848373 updated 0.429513848373
Delta2_star = 0.496332853741 updated 0.496332853741
Delta2_star = 0.42778321836 updated 0.42778321836
Delta2_star = 0.314911349651 updated 0.314911349651
Delta2_star = 0.329944674061 update

array([[ 0.38697897,  0.80594256],
       [ 0.69602493,  0.29357303],
       [ 0.31590602,  0.10924304],
       [ 0.45727168,  0.73815185],
       [ 0.89176841,  0.65807866],
       [ 0.38254531,  0.85446581],
       [ 0.13912346,  0.8975447 ],
       [ 0.95327977,  0.12563915],
       [ 0.06759128,  0.23241562],
       [ 0.9657017 ,  0.90792594],
       [ 0.04533807,  0.1889988 ],
       [ 0.74720951,  0.79611501],
       [ 0.2794845 ,  0.81914046],
       [ 0.07698606,  0.07443619],
       [ 0.64756924,  0.07949676],
       [ 0.98166427,  0.85329179],
       [ 0.63891609,  0.11039567],
       [ 0.07455675,  0.38665779],
       [ 0.14972337,  0.68214629],
       [ 0.61511681,  0.54130465]])

In [51]:
# setup sampler
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=[like,linP_Mpc_params])

In [ ]:
# burn-in phase
pos, prob, state = sampler.run_mcmc(p0, 100)
print('finished burn-in at',pos,prob)

Delta2_star = 0.377395794668 updated 0.377395794668
Delta2_star = 0.439204985912 updated 0.439204985912
Delta2_star = 0.363181204043 updated 0.363181204043
Delta2_star = 0.39145433607 updated 0.39145433607
Delta2_star = 0.478353682974 updated 0.478353682974
Delta2_star = 0.376509062805 updated 0.376509062805
Delta2_star = 0.327824692543 updated 0.327824692543
Delta2_star = 0.490655954803 updated 0.490655954803
Delta2_star = 0.313518256923 updated 0.313518256923
Delta2_star = 0.493140339774 updated 0.493140339774
Delta2_star = 0.309067614221 updated 0.309067614221
Delta2_star = 0.44944190211 updated 0.44944190211
Delta2_star = 0.355896899486 updated 0.355896899486
Delta2_star = 0.315397211784 updated 0.315397211784
Delta2_star = 0.429513848373 updated 0.429513848373
Delta2_star = 0.496332853741 updated 0.496332853741
Delta2_star = 0.42778321836 updated 0.42778321836
Delta2_star = 0.314911349651 updated 0.314911349651
Delta2_star = 0.329944674061 updated 0.329944674061
Delta2_star = 0.42

In [ ]:
# reset and run actual chains
sampler.reset()
nsteps=500
for i, result in enumerate(sampler.sample(pos, iterations=nsteps)):
    if i % 100 == 0:
        print(i,result[0])

Delta2_star = 0.317353712815 updated 0.317353712815
Delta2_star = 0.301734644678 updated 0.301734644678
Delta2_star = 0.355234603279 updated 0.355234603279
Delta2_star = 0.401288236847 updated 0.401288236847
Delta2_star = 0.45235111471 updated 0.45235111471
Delta2_star = 0.369634398537 updated 0.369634398537
Delta2_star = 0.307609713451 updated 0.307609713451
Delta2_star = 0.326929722332 updated 0.326929722332
Delta2_star = 0.462206792777 updated 0.462206792777
Delta2_star = 0.490317320416 updated 0.490317320416
Delta2_star = 0.497080448896 updated 0.497080448896
Delta2_star = 0.373454517535 updated 0.373454517535
Delta2_star = 0.406370894045 updated 0.406370894045
Delta2_star = 0.456850843526 updated 0.456850843526
Delta2_star = 0.467308095211 updated 0.467308095211
Delta2_star = 0.411797544178 updated 0.411797544178
Delta2_star = 0.459249702162 updated 0.459249702162
Delta2_star = 0.459280792047 updated 0.459280792047
Delta2_star = 0.456109254815 updated 0.456109254815
Delta2_star = 

In [ ]:
for i in range(ndim):
    plt.figure()
    plt.hist(sampler.flatchain[:,i], 100, color="k", histtype="step")
    plt.title("Dimension {0:d}".format(i))

In [ ]:
print("Mean acceptance fraction: {0:.3f}".format(np.mean(sampler.acceptance_fraction)))

### Use likelihood object to make several useful plots

In [ ]:
for p in like.free_params:
    print(p.name,p.value)

In [ ]:
z=np.linspace(2.0,5.0,100)
mf=like.theory.mf_model.get_mean_flux(z)
plt.plot(z,mf)
plt.xlabel('z')
plt.ylabel('<F>')

In [ ]:
like.plot_p1d()

In [ ]:
like.overplot_emulator_calls('gamma','sigT_Mpc',linP_Mpc_params=linP_Mpc_params)

In [ ]:
like.overplot_emulator_calls('mF','gamma',linP_Mpc_params=linP_Mpc_params)

In [ ]:
like.overplot_emulator_calls('Delta2_p','mF',linP_Mpc_params=linP_Mpc_params)

In [ ]:
like.overplot_emulator_calls('alpha_p','n_p',linP_Mpc_params=linP_Mpc_params)

In [ ]:
like.theory.cosmo.linP_model.linP_params